# Set up dataframe

In [3]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import kurtosis
from scipy.integrate import odeint
import statistics
from scipy.stats import lognorm
import math
import string
import sdeint
from joblib import Parallel, delayed 
import multiprocessing
from scipy.optimize import curve_fit
import pandas as pd
import seaborn as sns
from scipy import optimize
from scipy.stats import expon
import matplotlib.ticker as mtick
from scipy.stats import rv_continuous
from sklearn.metrics import mean_squared_error, r2_score 
from sympy import *
from sympy import summation, symbols, solve, Function, Sum, log
from numpy import random
from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,mark_inset)
from collections import Counter
import os

## 1.1) Set up functions

In [5]:
#function for the q-exponential function
def q_exp_func(x,q,l):
    #q = q_value
    return (2 - q) * l * np.sign(1 + (q- 1) * l * x)* (np.abs(1 + (q- 1) * l * x))**( 1/(1 - q))

#function for the exponential function
def exp_func(x,l):
    return l*np.exp(-l*x)   

#class for generating instances of the q-exponential function
class qExp_gen(rv_continuous): 
    "q exponential"
    #pdf defines the pdf
    def _pdf(self, x, l , q):
        self.l=l;
        self.q=q  
        if q==1: 
            return exp_func(x,l)
        else:
            return q_exp_func(x,q,l)

    def _stats(self,l, q):
        return [self.l,self.q,0,0]
    #fitstart provides a starting point for any MLE fit
    def _fitstart(self,data):
        return (1.1,1.1)
    #argcheck tests that the parameters are meaningful
    def _argcheck(self, l, q):
        return (l>0)&(q>0)
    
# check if a string is NAN   
def isNaN(string):
    return string != string

#read sites meta
europe_meta=pd.read_csv('data_sites.csv')

## 1.2 ) Define a function which match site area and type

In [ ]:
#enter site code
def type_area_sort(x):
    site_area = df[df["code"]==x]['area']
    site_type = df[df["code"]==x]['type']
    if site_area.to_string(index = False) in [" rural"," rural_remote"," rural_regional"," rural_nearcity"]:
        if site_type.to_string(index = False) in [" background"]:
            return "rural background"
        elif site_type.to_string(index = False) in [" traffic"]:
            return "suburban/rural traffic"
        elif site_type.to_string(index = False) in [" industrial"]:
            return "suburban/rural industrial"
        else:
            return np.nan
    elif site_area.to_string(index = False) in [" urban"]:        
        if site_type.to_string(index = False) in [" background"]:
            return "urban background"
        elif site_type.to_string(index = False) in [" traffic"]:
            return "urban traffic"
        elif site_type.to_string(index = False) in [" industrial"]:
            return "urban industrial"
        else:
            return np.nan
    elif site_area.to_string(index = False) in [" suburban"]:        
        if site_type.to_string(index = False) in [" background"]:
            return "suburban background"
        elif site_type.to_string(index = False) in [" traffic"]:
            return "suburban/rural traffic"
        elif site_type.to_string(index = False) in [" industrial"]:
            return "suburban/rural industrial"
        else:
            return np.nan
    else:
        return np.nan

## 1.3 ) Define a function for computing mean and standard deviation

In [ ]:
def statistics_output(code,pol):
    #check if the location has name
    if isNaN(europe_meta[europe_meta['site']==code]['site_name']).bool():
        return (np.nan,np.nan)
    #check if the location can be found on map
    elif math.isnan(europe_meta[europe_meta['site']==code]['latitude']):
        return (np.nan,np.nan)
    elif math.isnan(europe_meta[europe_meta['site']==code]['longitude']):
        return (np.nan,np.nan)
    #check if the area type is defined
    elif isNaN(europe_meta[europe_meta['site']==code]['site_type']).bool():
        return (np.nan,np.nan)
    elif isNaN(europe_meta[europe_meta['site']==code]['site_area']).bool():
        return (np.nan,np.nan)   
    else:
        #Choose 2017-2021 data
        importdata = pd.read_csv(str(code)+'.csv')
        if (importdata.empty):
            return (np.nan,np.nan)
        else:
            pollutant=importdata["variable"].drop_duplicates()
            if pol in pollutant.tolist():
                importdata_pol = importdata[importdata["variable"]==pol]
                #check the unit to be ug.m-3
                importdata_pol=importdata_pol[importdata_pol["unit"]=='ug.m-3']
                #remove zeros and negative values
                importdata_pol=importdata_pol[importdata_pol["value"]>0]
                #Check if the amount of dataset is fewer than a year
                if len(importdata_pol)<365*24:
                    return (np.nan,np.nan)
                else:                   
                    len_validity = len(importdata_pol[importdata_pol["validity"]>1])                  
                    # Check if censored data consist 15% of the entire dataset
                    if len_validity/len(importdata_pol)>0.15:
                        return (np.nan,np.nan)                                                 
                    else:
                        data1 = importdata_pol["value"]
                        data2 = Counter(data1)
                        # Check if the most occured data consist 15% of the entire dataset
                        if data2.most_common(1)[0][1]/len(data1)>0.15:
                            return (np.nan,np.nan)  
                        else:                           
                            data_full=np.array(data1)                           
            else:
                return (np.nan,np.nan)     
        #plot histogram    
        datarange=1.5*np.mean(data_full)
        data_close_to_peak=data_full[data_full<datarange]
        hist_lines=sns.distplot(data_close_to_peak, norm_hist=True, kde=True, color='gold',
                                kde_kws={"bw":0.2,"gridsize":max(500,int(10*datarange))});
        (xvalues_hist,yvalues_hist)=hist_lines.get_lines()[0].get_data()
        #define the concentration which appears the most
        peakOfdata=xvalues_hist[np.where(yvalues_hist==max(yvalues_hist))[0][0]]
        #only consider the data higher than the peak 
        data_tail=data_full[data_full>=peakOfdata]
        plt.clf()        
        mslist=(np.mean(data_tail),np.std(data_tail))
        return mslist

## 1.4) Define a function which outputs q-value

In [ ]:
def q_value_output(code,pol):
    #check if the location has name
    if isNaN(europe_meta[europe_meta['site']==code]['site_name']).bool():
        return (np.nan,np.nan)
    #check if the location can be found on map
    elif math.isnan(europe_meta[europe_meta['site']==code]['latitude']):
        return (np.nan,np.nan)
    elif math.isnan(europe_meta[europe_meta['site']==code]['longitude']):
        return (np.nan,np.nan)
    #check if the area type is defined
    elif isNaN(europe_meta[europe_meta['site']==code]['site_type']).bool():
        return (np.nan,np.nan)
    elif isNaN(europe_meta[europe_meta['site']==code]['site_area']).bool():
        return (np.nan,np.nan)   
    else:
        #Choose 2017-2021 data
        importdata = pd.read_csv(str(code)+'.csv')
        if (importdata.empty):
            return (np.nan,np.nan)
        else:
            pollutant=importdata["variable"].drop_duplicates()
            if pol in pollutant.tolist():
                importdata_pol = importdata[importdata["variable"]==pol]
                #check the unit to be ug.m-3
                importdata_pol=importdata_pol[importdata_pol["unit"]=='ug.m-3']
                #remove zeros and negative values
                importdata_pol=importdata_pol[importdata_pol["value"]>0]
                #Check if the amount of dataset is fewer than a year
                if len(importdata_pol)<365*24:
                    return (np.nan,np.nan)
                else:                   
                    len_validity = len(importdata_pol[importdata_pol["validity"]>1])                  
                    # Check if censored data consist 15% of the entire dataset
                    if len_validity/len(importdata_pol)>0.15:
                        return (np.nan,np.nan)                                                 
                    else:
                        data1 = importdata_pol["value"]
                        data2 = Counter(data1)
                        # Check if the most occured data consist 15% of the entire dataset
                        if data2.most_common(1)[0][1]/len(data1)>0.15:
                            return (np.nan,np.nan)  
                        else:                           
                            data_full=np.array(data1)                           
            else:
                return (np.nan,np.nan)                   
                
    #plot histograms
    datarange=1.5*np.mean(data_full)
    data_close_to_peak=data_full[data_full<datarange]
    hist_lines=sns.distplot(data_close_to_peak, norm_hist=True, kde=True, color='gold',
                            kde_kws={"bw":0.2,"gridsize":max(500,int(10*datarange))});
    (xvalues_hist,yvalues_hist)=hist_lines.get_lines()[0].get_data()
    #define the concentration which appears the most
    peakOfdata=xvalues_hist[np.where(yvalues_hist==max(yvalues_hist))[0][0]]
    #only consider the data higher than the peak
    data_tail=data_full[data_full>peakOfdata]
    plt.clf();

    #generate an instance of the q-exponential, we need a=0 as lower bound for the support of the function            
    q_exp=qExp_gen(name="q exponential",a=0)    
    #MLE for q-exponential distribution (built in function)    
    parameters_Qexp_MLE=q_exp.fit(data_tail,1.1,1.1,floc=peakOfdata,fscale=1)
    return parameters_Qexp_MLE

## 1.5) Define a function for classifying wind force scale

In [ ]:
def ws_sort(code,pol):
    #check if the location has name
    if isNaN(europe_meta[europe_meta['site']==code]['site_name']).bool():
        return (np.nan)
    else:
        #Enter the path of the folder of your svaed data
        path = 'Enter the path of your saved data here'+str(code)+'.csv'
        # path exists or not
        if os.path.exists(path):
            #Choose 2017-2021 data
            importdata = pd.read_csv(path)
            if (importdata.empty):
                return (np.nan)
            else:
                pollutant=importdata["variable"].drop_duplicates()
                if pol in pollutant.tolist() and 'ws' in importdata.columns:
                    #choose pollutant
                    importdata_pol = importdata[importdata["variable"]==pol]
                    #only wind speed data corresponding to valid concentration data
                    importdata_ws = importdata_pol.loc[importdata_pol['value']>0, 'ws']
                    #remove negative values
                    importdata_ws=importdata_ws[importdata_ws>=0]
                    #remove NAN data
                    #importdata_ws = [x for x in importdata_ws if str(x) != 'nan']
                    #Check if the amount of dataset is fewer than a year
                    if len(importdata_ws)<365*24:
                        return (np.nan)
                    else:
                        data_full=np.array(importdata_ws)
                        data_mean=np.mean(data_full)
                        #print(data_mean)
                        if data_mean<1.6:
                            return "Calm & Light Air"
                        elif data_mean>=1.6 and data_mean<3.4:
                            return "Light Breeze"
                        elif data_mean>=3.4 and data_mean<5.5:
                            return "Gentle Breeze"
                        elif data_mean>=5.5 and data_mean<8:
                            return "Moderate breeze"
                        elif data_mean>=8 and data_mean<10.8:
                            return "Fresh breeze"
                        elif data_mean>=10.8 and data_mean<13.9:
                            return "Strong breeze"
                        elif data_mean>=13.9 and data_mean<17.2:
                            return "Near Gale"
                        elif data_mean>=17.2 and data_mean<20.8:
                            return "Gale"
                        elif data_mean>=20.8 and data_mean<24.5:
                            return "Strong Gale"                       
                        elif data_mean>=24.5 and data_mean<28.5:
                            return "Storm"
                        elif data_mean>=28.5 and data_mean<32.7:
                            return "Violent storm"
                        elif data_mean>=32.7:
                            return "Hurricane"                      
                        else:
                            return (np.nan) 
                        
                else:
                    return (np.nan)  
        else:
            return (np.nan)

## 1.6) Define a function which outputs log-likelihood

In [ ]:
#choose the site code and pollutant type
def log_likelihood_output(code,pol):
    importdata = pd.read_csv(str(code)+'.csv')
    importdata_pol = importdata[importdata["variable"]==pol]
    importdata_pol=importdata_pol[importdata_pol["unit"]=='ug.m-3']
    importdata_pol=importdata_pol[importdata_pol["value"]>0]
    data1 = importdata_pol["value"]
    data_full=np.array(data1)                           
    
    #plot histograms
    datarange=1.5*np.mean(data_full)
    data_close_to_peak=data_full[data_full<datarange]
    hist_lines=sns.distplot(data_close_to_peak, norm_hist=True, kde=True, kde_kws={"bw":0.2,"gridsize":500});
    (xvalues_hist,yvalues_hist)=hist_lines.get_lines()[0].get_data()
    #define the concentration which appears the most
    peakOfdata=xvalues_hist[np.where(yvalues_hist==max(yvalues_hist))[0][0]]
    #only consider the data higher than the peak
    data_tail=data_full[data_full>=peakOfdata]
    plt.clf();

    #generate an instance of the q-exponential, we need a=0 as lower bound for the support of the function            
    q_exp=qExp_gen(name="q exponential",a=0)  #a=peakofdata      
    #MLE for q-exponential distribution (built in function)  #try different q,l   
    parameters_Qexp_MLE=q_exp.fit(data_tail,1.1,1.1,floc=peakOfdata,fscale=1)  
    #MLE for log-normal distribution 
    lognorm_param = stats.lognorm.fit(data_full, floc=0)
    #weibull fit
    weibull_param = stats.weibull_min.fit(data_full, floc=0)
    #gamma fit
    gamma_param=stats.gamma.fit(data_full,floc=0)
    
    #compute log-likelihood
    q_likelihood = 0
    l_likelihood = 0 
    w_likelihood = 0
    g_likelihood = 0    
    
    for item1 in data_tail:      
        #choose q-value
        q_likelihood = np.log(q_exp_func((item1-peakOfdata),parameters_Qexp_MLE[1],parameters_Qexp_MLE[0]))
        q_likelihood += q_likelihood 
        #lognorm
        l_likelihood = np.log(len(data_full)/len(data_tail)*lognorm.pdf(
            item1,lognorm_param[0],lognorm_param[1],lognorm_param[2]))
        l_likelihood += l_likelihood
        #weibull
        w_likelihood = np.log(len(data_full)/len(data_tail)*stats.weibull_min.pdf(
            item1,weibull_param[0],loc = weibull_param[1], scale=weibull_param[2]))
        w_likelihood += w_likelihood
        #gamma
        g_likelihood = np.log(len(data_full)/len(data_tail)*stats.gamma.pdf(
            item1, gamma_param[0], loc=gamma_param[1], scale=gamma_param[2]))
        g_likelihood += g_likelihood                 
        
    log_likelihood = (q_likelihood,l_likelihood,w_likelihood,g_likelihood)
    return log_likelihood


## 2.1) set up dataframe for area type

In [ ]:
#Read the european locations
europe_meta=pd.read_csv('data_sites.csv')
df = pd.DataFrame({'site':europe_meta['site_name'],'code':europe_meta['site'],'type':europe_meta['site_type'],'area':europe_meta['site_area']})
typelist=[]
for x in europe_meta['site']:        
    typelist.append(type_area_sort(x))    
df["area_type"] = typelist
df.to_csv('European area type.csv', index=False)

## 2.2) set up dataframe for mean and standard deviation

In [ ]:
statistics_df = df
mean_list=[]
std_list=[]
for x in europe_meta['site']:
    mslist=statistics_output(x,'no')
    mean_list.append(mslist[0])
    std_list.append(mslist[1])   
statistics_df["mean"] = mean_list
statistics_df["std"] = std_list
statistics_df.to_csv('europe no mean versus standard deviation.csv', index=False)


In [ ]:
statistics_df = df
mean_list=[]
std_list=[]
for x in europe_meta['site']:
    mslist=statistics_output(x,'no2')
    mean_list.append(mslist[0])
    std_list.append(mslist[1])   
statistics_df["mean"] = mean_list
statistics_df["std"] = std_list
statistics_df.to_csv('europe no2 mean versus standard deviation.csv', index=False)


In [ ]:
statistics_df = df
mean_list=[]
std_list=[]
for x in europe_meta['site']:
    mslist=statistics_output(x,'pm2.5')
    mean_list.append(mslist[0])
    std_list.append(mslist[1])   
statistics_df["mean"] = mean_list
statistics_df["std"] = std_list
statistics_df.to_csv('europe pm2.5 mean versus standard deviation.csv', index=False)


In [ ]:
statistics_df = df
mean_list=[]
std_list=[]
for x in europe_meta['site']:
    mslist=statistics_output(x,'pm10')
    mean_list.append(mslist[0])
    std_list.append(mslist[1])   
statistics_df["mean"] = mean_list
statistics_df["std"] = std_list
statistics_df.to_csv('europe pm10 mean versus standard deviation.csv', index=False)


## 2.3) set up dataframe for lambda and q-value

In [ ]:
#Read the european locations
europe_meta=pd.read_csv('data_sites.csv')
qlist=[]
llist=[]
qvalue_df = df
for x in europe_meta['site']:        
    parameters_Qexp_MLE = q_value_output(x,'no')
    qlist.append(parameters_Qexp_MLE[1])
    llist.append(parameters_Qexp_MLE[0])   
qvalue_df["q"] = qlist
qvalue_df["l"] = llist
qvalue_df.to_csv('no q,lamda dataframe.csv', index=False)

In [ ]:
qlist=[]
llist=[]
qvalue_df = df
for x in europe_meta['site']:        
    parameters_Qexp_MLE = q_value_output(x,'no2')
    qlist.append(parameters_Qexp_MLE[1])
    llist.append(parameters_Qexp_MLE[0])   
qvalue_df["q"] = qlist
qvalue_df["l"] = llist
qvalue_df.to_csv('no2 q,lamda dataframe.csv', index=False)


In [ ]:
qlist=[]
llist=[]
qvalue_df = df
for x in europe_meta['site']:        
    parameters_Qexp_MLE = q_value_output(x,'pm2.5')
    qlist.append(parameters_Qexp_MLE[1])
    llist.append(parameters_Qexp_MLE[0])   
qvalue_df["q"] = qlist
qvalue_df["l"] = llist
qvalue_df.to_csv('pm2.5 q,lamda dataframe.csv', index=False)

In [ ]:
qlist=[]
llist=[]
qvalue_df = df
for x in europe_meta['site']:        
    parameters_Qexp_MLE = q_value_output(x,'pm10')
    qlist.append(parameters_Qexp_MLE[1])
    llist.append(parameters_Qexp_MLE[0])   
qvalue_df["q"] = qlist
qvalue_df["l"] = llist
qvalue_df.to_csv('pm10 q,lamda dataframe.csv', index=False)


## 2.4) set up dataframe for wind

In [ ]:
#NO
ws_typelist=[]
for x in europe_meta['site']:        
    ws_typelist.append(ws_sort(x,'no'))    
no_qvalue_df["ws_type"] = ws_typelist
no_qvalue_df.to_csv('no ws dataframe.csv', index=False)

In [ ]:
#NO2
ws_typelist=[]
for x in europe_meta['site']:        
    ws_typelist.append(ws_sort(x,'no2'))    
no2_qvalue_df["ws_type"] = ws_typelist
no2_qvalue_df.to_csv('no2 ws dataframe.csv', index=False)

In [ ]:
#PM2.5
ws_typelist=[]
for x in europe_meta['site']:        
    ws_typelist.append(ws_sort(x,'pm2.5'))    
pm25_qvalue_df["ws_type"] = ws_typelist
pm25_qvalue_df.to_csv('pm2.5 ws dataframe.csv', index=False)

In [ ]:
#PM10
ws_typelist=[]
for x in europe_meta['site']:        
    ws_typelist.append(ws_sort(x,'pm10'))    
pm10_qvalue_df["ws_type"] = ws_typelist
pm10_qvalue_df.to_csv('pm10 ws dataframe.csv', index=False)

## 2.5) set up dataframe for log-likelihood

In [ ]:
# NO
q_likelihood_list=[]
l_likelihood_list=[]
w_likelihood_list=[]
g_likelihood_list=[]

likelihood_df = df
for x in europe_meta['site']:        
    log_likelihood = log_likelihood_output(x,'no')
    q_likelihood_list.append(log_likelihood[0])
    l_likelihood_list.append(log_likelihood[1])
    w_likelihood_list.append(log_likelihood[2])
    g_likelihood_list.append(log_likelihood[3])    
likelihood_df["q_ll"] = q_likelihood_list
likelihood_df["l_ll"] = l_likelihood_list
likelihood_df["w_ll"] = w_likelihood_list
likelihood_df["g_ll"] = g_likelihood_list

likelihood_df.to_csv('no likelihood dataframe.csv', index=False)

In [ ]:
#NO2
q_likelihood_list=[]
l_likelihood_list=[]
w_likelihood_list=[]
g_likelihood_list=[]

likelihood_df = df
for x in europe_meta['site']:        
    log_likelihood = log_likelihood_output(x,'no2')
    q_likelihood_list.append(log_likelihood[1])
    l_likelihood_list.append(log_likelihood[2])
    w_likelihood_list.append(log_likelihood[3])
    g_likelihood_list.append(log_likelihood[4])    
likelihood_df["q_ll"] = q_likelihood_list
likelihood_df["l_ll"] = l_likelihood_list
likelihood_df["w_ll"] = w_likelihood_list
likelihood_df["g_ll"] = g_likelihood_list

likelihood_df.to_csv('no2 likelihood dataframe.csv', index=False)

In [ ]:
#PM2.5
q_likelihood_list=[]
l_likelihood_list=[]
w_likelihood_list=[]
g_likelihood_list=[]

likelihood_df = df
for x in europe_meta['site']:        
    log_likelihood = log_likelihood_output(x,'pm2.5')
    q_likelihood_list.append(log_likelihood[0])
    l_likelihood_list.append(log_likelihood[1])
    w_likelihood_list.append(log_likelihood[2])
    g_likelihood_list.append(log_likelihood[3])     
likelihood_df["q_ll"] = q_likelihood_list
likelihood_df["l_ll"] = l_likelihood_list
likelihood_df["w_ll"] = w_likelihood_list
likelihood_df["g_ll"] = g_likelihood_list

likelihood_df.to_csv('pm2.5 likelihood dataframe.csv', index=False)

In [ ]:
#PM10
q_likelihood_list=[]
l_likelihood_list=[]
w_likelihood_list=[]
g_likelihood_list=[]

likelihood_df = df
for x in europe_meta['site']:        
    log_likelihood = log_likelihood_output(x,'pm10')
    q_likelihood_list.append(log_likelihood[0])
    l_likelihood_list.append(log_likelihood[1])
    w_likelihood_list.append(log_likelihood[2])
    g_likelihood_list.append(log_likelihood[3])    
likelihood_df["q_ll"] = q_likelihood_list
likelihood_df["l_ll"] = l_likelihood_list
likelihood_df["w_ll"] = w_likelihood_list
likelihood_df["g_ll"] = g_likelihood_list

likelihood_df.to_csv('pm10 likelihood dataframe.csv', index=False)